# Matrixmultiplkationsverfahren

Wir werden uns beschäftigen mit Matrixmultiplication so wie gelernt in LinAlg 1 und ein neues Verfahren betrachten, dass theoretisch schneller sein soll

## LinAlg 1 style

In LinAlg 1 haben wir folgendes Verfahren gelernt um das Matrixprodukt $AB$ von einer $m \times n$ Matrix $A = (a_{ij})$ und $n \times k$ Matrix $B = (b_{ij})$ aus zu rechnen:

$$(AB)_{ij} = \sum_{p=1}^n a_{ip}b_{pj}$$

Eine bemerksame Person sieht, dass $mnk$ Multiplikationen notwendig sind um das Produkt aus zu rechnen: $n$ um $(AB)_{ij}$ aus zu rechnen und $m$ und $n$ möglichkeiten für $i$ bezüglich $j$.



In [ ]:
from time import sleep
delay = 0.001
def multiply(a,b,d):
    sleep(float(d))
    return a*b

def linalg1multiply(A,B):
    if (A.ncols() != B.nrows()):
        print('dimensions disagree')
        return
    AB = matrix(RR, A.nrows(), B.ncols())
    for i in range(A.nrows()):
        for j in range(B.ncols()):
            for k in range(A.ncols()):
                AB[i,j] += multiply(A[i,k],B[k,j],delay)
                # AB[i,j] += A[i,k]*B[k,j]
    return AB

A = random_matrix(RR,5,6)
B = random_matrix(RR,6,4)

print(linalg1multiply(A,B) - A*B)

## Strassen style

Im Jahr 1969 ist es Volker Strassen aufgefallen, dass optimaler kann. Obwohl es uns soweit viel Spass gemacht hat, kann mann sich vorstellen, dass es langweilig ist um sehr grosse Matrizen miteinander zu multiplizieren. Computer haben ungefär dasselbe; es ist 'teuerer' für einen Computer um zwei zahlen miteinander zu multiplizieren, als sie zu addieren. Jetzt schauwen wir mal die nächste Matrizen an:

$$A = \begin{pmatrix}a_{11} & a_{12}\\ a_{21} & a_{22}\end{pmatrix}, \quad B = \begin{pmatrix}b_{11} & b_{12}\\ b_{21} & b_{22}\end{pmatrix}$$

und wir definieren

\begin{align*}
s_1&= (a_{11} + a_{22})(b_{11} + b_{22})\\
s_2 &= (a_{21} + a_{22})b_{11}\\
s_3 &= a_{11}(b_{12} - b_{22})\\
s_4 &= a_{22}(b_{21} - b_{11})\\
s_5 &= (a_{11} + a_{12})b_{22}\\
s_6 &= (a_{21} - a_{11})(b_{11} + b_{12})\\
s_7 &= (a_{12} - a_{22})(b_{21} + b_{22})
\end{align*}

Jeztz setzen wir

$$S = \begin{pmatrix}s_1 + s_4 -  s_5 + s_7 & s_3 + s_5 \\ s_2 + s_4 & s_1 - s_2 + s_3 + s_6\end{pmatrix}$$

und überpfüfen dass $S = AB$. Die benötigte Anzahl der Multiplikationen ist $7$ und das ist weniger als $8$! Dies meint dass für $2^n \times 2^n$ Matrizen, dieses Verfahren nur $O(N^{\log(7)}) \approx O(N^{2.8074})$ multiplkationen braucht stat $O(N^3)$, wobei $N = 2^n$



In [ ]:
def strassenmultiply(A,B):
    if (A.nrows() == 1 | B.ncols() == 1):
        return multiply(A,B,delay)
        # return A*B
    d = A.nrows() / 2
    A11 = A.submatrix(0,0,d,d)
    A21 = A.submatrix(d,0,-1,d)
    A12 = A.submatrix(0,d,d,-1)
    A22 = A.submatrix(d,d,-1,-1)

    B11 = B.submatrix(0,0,d,d)
    B21 = B.submatrix(d,0,-1,d)
    B12 = B.submatrix(0,d,d,-1)
    B22 = B.submatrix(d,d,-1,-1)
    s1 = strassenmultiply(A11 + A22, B11 + B22)
    s2 = strassenmultiply(A21 + A22, B11)
    s3 = strassenmultiply(A11, B12 - B22)
    s4 = strassenmultiply(A22, B21 - B11)
    s5 = strassenmultiply(A11 + A12, B22)
    s6 = strassenmultiply(A21 - A11, B11 + B12)
    s7 = strassenmultiply(A12 - A22, B21 + B22)
    return block_matrix([[s1 + s4 - s5 + s7, s3 + s5], [s2 + s4, s1 - s2 + s3 + s6]])

C = random_matrix(RR, 2^5)
D = random_matrix(RR, 2^5)
#print(norm(strassenmultiply(C,D) - C*D))

w = walltime()
linalg1multiply(C,D)
w1 = walltime(w)
w = walltime()
strassenmultiply(C,D)
w2 = walltime(w)
print('LinAlg Multiplikation dauert', w1, 'Sekunden')
print('Strassen Multiplikation dauert', w2, 'Sekunden')